# Chapter 19: 特殊方法與運算子重載
# Part I: 理論基礎

## 📋 本章概覽

### 學習目標
- 理解特殊方法（magic methods）的運作原理
- 學會使用運算子重載讓自訂類別更自然
- 掌握上下文管理器協定
- 實作完整的容器類別

### 先備知識
- Chapter 16-18（類別、封裝、繼承）

### 預計時長
- 90 分鐘（理論 + 範例）

## 🔑 核心概念

### 什麼是特殊方法？

特殊方法（Magic Methods / Dunder Methods）是 Python 中以雙底線（`__`）開頭和結尾的方法。

它們讓你的自訂類別能像內建型別一樣使用：
- `obj1 + obj2` → 呼叫 `obj1.__add__(obj2)`
- `len(obj)` → 呼叫 `obj.__len__()`
- `str(obj)` → 呼叫 `obj.__str__()`

### 為什麼需要它們？

**設計理念**：讓自訂類別看起來像內建型別，提供一致的使用體驗。

# Part II: 實作演練

## 範例 1：`__init__` 與 `__str__` 基礎

最基本的特殊方法：初始化與字串表示

In [ ]:
class Person:
    """人員類別 - 示範基本特殊方法"""
    
    def __init__(self, name, age):
        """初始化方法：建立物件時自動呼叫"""
        self.name = name
        self.age = age
    
    def __str__(self):
        """字串表示：給使用者看的，易讀"""
        return f"{self.name}（{self.age} 歲）"

# 測試
p = Person("Alice", 25)
print(f"建立物件：{p}")  # 呼叫 __str__
print(f"型別：{type(p)}")

In [ ]:
# 沒有 __str__ 的情況
class PersonBad:
    def __init__(self, name, age):
        self.name = name
        self.age = age

p_bad = PersonBad("Bob", 30)
print(p_bad)  # 輸出不友善的記憶體位址

**重點說明**：
- `__init__`：物件建立時呼叫，用於初始化屬性
- `__str__`：`print()` 或 `str()` 時呼叫，返回使用者友善的字串
- 沒有 `__str__` 時，Python 使用預設格式（記憶體位址）

## 範例 2：`__repr__` 的正確使用

`__repr__` 是給開發者看的，應該包含完整資訊

In [ ]:
class Book:
    """書籍類別 - 示範 __str__ 與 __repr__ 的差異"""
    
    def __init__(self, title, author, year):
        self.title = title
        self.author = author
        self.year = year
    
    def __str__(self):
        """給讀者看：簡潔易讀"""
        return f"《{self.title}》- {self.author}"
    
    def __repr__(self):
        """給程式設計師看：完整且可重建"""
        return f"Book('{self.title}', '{self.author}', {self.year})"

# 測試
book = Book("Python 進階教學", "張三", 2025)

print("使用 str():")
print(str(book))  # 呼叫 __str__

print("\n使用 repr():")
print(repr(book))  # 呼叫 __repr__

print("\n在列表中（使用 __repr__）:")
books = [book]
print(books)

In [ ]:
# __repr__ 的黃金法則：eval(repr(obj)) 應該能重建物件
book_repr = repr(book)
print(f"repr 結果：{book_repr}")

# 使用 eval 重建物件（實務上少用，但這是設計目標）
book_copy = eval(book_repr)
print(f"重建的物件：{book_copy}")

**設計原則**：
- `__str__`：給終端使用者，專注於可讀性
- `__repr__`：給開發者，專注於完整性與可重建性
- 只實作一個時，選擇 `__repr__`（Python 會 fallback）
- 理想情況：`eval(repr(obj)) == obj`

## 範例 3：`__len__` 與 `__getitem__`（序列協定）

實作這兩個方法，你的類別就能像列表一樣使用

In [ ]:
class Playlist:
    """播放清單類別 - 示範序列協定"""
    
    def __init__(self, name):
        self.name = name
        self.songs = []  # 內部儲存歌曲
    
    def add_song(self, song):
        """新增歌曲"""
        self.songs.append(song)
    
    def __len__(self):
        """返回歌曲數量"""
        return len(self.songs)
    
    def __getitem__(self, index):
        """支援索引存取"""
        return self.songs[index]
    
    def __str__(self):
        return f"播放清單：{self.name}（{len(self)} 首歌）"

# 測試
playlist = Playlist("我的最愛")
playlist.add_song("歌曲 A")
playlist.add_song("歌曲 B")
playlist.add_song("歌曲 C")

print(playlist)
print(f"總共有 {len(playlist)} 首歌")  # 使用 len()
print(f"第一首：{playlist[0]}")  # 使用索引
print(f"最後一首：{playlist[-1]}")  # 負數索引也可以！

In [ ]:
# 支援切片（因為有 __getitem__）
print("\n前兩首歌：")
for song in playlist[0:2]:
    print(f"  - {song}")

# 支援迭代（因為有 __getitem__ 和 __len__）
print("\n所有歌曲：")
for song in playlist:
    print(f"  ♪ {song}")

**重點說明**：
- `__len__`：讓物件支援 `len()` 函式
- `__getitem__`：讓物件支援 `obj[index]` 存取
- 實作這兩個方法，自動獲得：
  - 切片功能
  - 迭代功能（for 迴圈）
  - 負數索引

## 範例 4：運算子重載 - 算術運算

讓自訂類別支援 +、-、* 等運算

In [ ]:
class Vector:
    """2D 向量類別 - 示範算術運算子重載"""
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def __add__(self, other):
        """向量相加：v1 + v2"""
        return Vector(self.x + other.x, self.y + other.y)
    
    def __sub__(self, other):
        """向量相減：v1 - v2"""
        return Vector(self.x - other.x, self.y - other.y)
    
    def __mul__(self, scalar):
        """純量乘法：v * 3"""
        return Vector(self.x * scalar, self.y * scalar)
    
    def __str__(self):
        return f"Vector({self.x}, {self.y})"
    
    def __repr__(self):
        return f"Vector({self.x}, {self.y})"

# 測試
v1 = Vector(1, 2)
v2 = Vector(3, 4)

print(f"v1 = {v1}")
print(f"v2 = {v2}")
print(f"v1 + v2 = {v1 + v2}")  # 向量相加
print(f"v2 - v1 = {v2 - v1}")  # 向量相減
print(f"v1 * 3 = {v1 * 3}")    # 純量乘法

In [ ]:
# 複合運算
result = (v1 + v2) * 2 - v1
print(f"(v1 + v2) * 2 - v1 = {result}")

# 驗證不可變性（不修改原物件）
v3 = Vector(5, 6)
v4 = v3 + Vector(1, 1)
print(f"\nv3 原本的值：{v3}")  # v3 沒有被修改
print(f"v4 新的值：{v4}")

**設計原則**：
- 算術運算子應該返回**新物件**，不修改原物件
- 只重載意義明確的運算子
- 運算行為應符合直覺（+ 表示相加，* 表示縮放等）

## 範例 5：比較運算子重載

讓物件可以比較大小、排序

In [ ]:
class Student:
    """學生類別 - 示範比較運算子重載"""
    
    def __init__(self, name, score):
        self.name = name
        self.score = score
    
    def __eq__(self, other):
        """相等比較：s1 == s2"""
        return self.score == other.score
    
    def __lt__(self, other):
        """小於比較：s1 < s2（用於排序）"""
        return self.score < other.score
    
    def __le__(self, other):
        """小於等於：s1 <= s2"""
        return self.score <= other.score
    
    def __gt__(self, other):
        """大於：s1 > s2"""
        return self.score > other.score
    
    def __str__(self):
        return f"{self.name}（{self.score} 分）"

# 測試
s1 = Student("Alice", 85)
s2 = Student("Bob", 92)
s3 = Student("Carol", 85)

print(f"s1 = {s1}")
print(f"s2 = {s2}")
print(f"s3 = {s3}")
print(f"\ns1 == s3: {s1 == s3}")  # 分數相同
print(f"s1 < s2: {s1 < s2}")
print(f"s2 > s1: {s2 > s1}")

In [ ]:
# 排序功能
students = [s2, s1, s3]
print("\n排序前：")
for s in students:
    print(f"  {s}")

students.sort()  # 使用 __lt__ 排序
print("\n排序後（由低到高）：")
for s in students:
    print(f"  {s}")

In [ ]:
# 使用 functools.total_ordering 簡化（只需定義 __eq__ 和一個比較方法）
from functools import total_ordering

@total_ordering
class StudentSimple:
    def __init__(self, name, score):
        self.name = name
        self.score = score
    
    def __eq__(self, other):
        return self.score == other.score
    
    def __lt__(self, other):
        return self.score < other.score
    
    # __le__, __gt__, __ge__ 會自動產生！
    
    def __str__(self):
        return f"{self.name}（{self.score} 分）"

# 測試自動產生的方法
s4 = StudentSimple("David", 88)
s5 = StudentSimple("Eve", 95)

print(f"\ns4 <= s5: {s4 <= s5}")  # 自動產生的方法
print(f"s5 >= s4: {s5 >= s4}")    # 自動產生的方法

**重點說明**：
- `__eq__`：定義相等性（==）
- `__lt__`：定義小於（<），也用於排序
- 使用 `@total_ordering` 裝飾器可減少程式碼（只需定義 `__eq__` 和一個比較方法）
- 實作比較方法後，物件可以排序、比較

## 範例 6：`__call__` - 可呼叫物件

實作 `__call__` 讓物件像函式一樣可以呼叫

In [ ]:
class Multiplier:
    """乘法器 - 示範可呼叫物件"""
    
    def __init__(self, factor):
        self.factor = factor
    
    def __call__(self, x):
        """讓物件可以像函式一樣呼叫"""
        return x * self.factor
    
    def __str__(self):
        return f"Multiplier(×{self.factor})"

# 測試
double = Multiplier(2)
triple = Multiplier(3)

print(f"double = {double}")
print(f"double(5) = {double(5)}")  # 像函式一樣呼叫
print(f"triple(5) = {triple(5)}")

In [ ]:
# 實用範例：計數器
class Counter:
    """呼叫計數器"""
    
    def __init__(self):
        self.count = 0
    
    def __call__(self):
        """每次呼叫計數加一"""
        self.count += 1
        return self.count
    
    def __str__(self):
        return f"Counter（已呼叫 {self.count} 次）"

counter = Counter()
print(f"第 {counter()} 次呼叫")  # 1
print(f"第 {counter()} 次呼叫")  # 2
print(f"第 {counter()} 次呼叫")  # 3
print(f"\n{counter}")

In [ ]:
# 實用範例：保存狀態的函式
class RunningAverage:
    """移動平均計算器"""
    
    def __init__(self):
        self.values = []
    
    def __call__(self, new_value):
        """加入新值並返回平均"""
        self.values.append(new_value)
        return sum(self.values) / len(self.values)

avg = RunningAverage()
print(f"加入 10，平均：{avg(10)}")
print(f"加入 20，平均：{avg(20)}")
print(f"加入 30，平均：{avg(30)}")
print(f"加入 40，平均：{avg(40)}")

**應用場景**：
- 需要保存狀態的「函式」
- 裝飾器（decorators）
- 回呼函式（callbacks）
- 函式式程式設計

## 範例 7：`__enter__` / `__exit__` - 上下文管理器

實作上下文管理器協定，支援 `with` 敘述

In [ ]:
class FileManager:
    """檔案管理器 - 示範上下文管理器"""
    
    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode
        self.file = None
    
    def __enter__(self):
        """進入 with 區塊時執行：開啟資源"""
        print(f"開啟檔案：{self.filename}")
        self.file = open(self.filename, self.mode)
        return self.file  # 返回值賦給 as 後的變數
    
    def __exit__(self, exc_type, exc_value, traceback):
        """離開 with 區塊時執行：關閉資源"""
        print(f"關閉檔案：{self.filename}")
        if self.file:
            self.file.close()
        
        # 返回 False 或 None：重新拋出例外
        # 返回 True：抑制例外
        return False

# 測試
with FileManager("test.txt", "w") as f:
    f.write("Hello, World!\n")
    f.write("這是測試檔案。")
    print("寫入完成")
# 離開 with 區塊後，__exit__ 自動執行

print("\n讀取檔案內容：")
with FileManager("test.txt", "r") as f:
    content = f.read()
    print(content)

In [ ]:
# 實用範例：計時器
import time

class Timer:
    """計時上下文管理器"""
    
    def __init__(self, name="程式碼區塊"):
        self.name = name
        self.start_time = None
    
    def __enter__(self):
        print(f"⏱️  開始計時：{self.name}")
        self.start_time = time.time()
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        elapsed = time.time() - self.start_time
        print(f"⏱️  結束計時：{self.name}，耗時 {elapsed:.4f} 秒")
        return False

# 測試
with Timer("迴圈計算"):
    total = 0
    for i in range(1000000):
        total += i
    print(f"計算結果：{total}")

In [ ]:
# 實用範例：資料庫連線管理
class DatabaseConnection:
    """模擬資料庫連線管理器"""
    
    def __init__(self, db_name):
        self.db_name = db_name
        self.connection = None
    
    def __enter__(self):
        print(f"🔌 連接到資料庫：{self.db_name}")
        self.connection = f"Connection to {self.db_name}"
        return self.connection
    
    def __exit__(self, exc_type, exc_value, traceback):
        print(f"🔌 關閉資料庫連接：{self.db_name}")
        self.connection = None
        
        # 處理例外
        if exc_type is not None:
            print(f"❌ 發生錯誤：{exc_value}")
        
        return False  # 重新拋出例外

# 正常使用
print("=== 正常情況 ===")
with DatabaseConnection("mydb") as conn:
    print(f"執行查詢：{conn}")

# 發生例外時
print("\n=== 例外情況 ===")
try:
    with DatabaseConnection("mydb") as conn:
        print(f"執行查詢：{conn}")
        raise ValueError("查詢失敗！")
except ValueError as e:
    print(f"捕獲例外：{e}")

print("\n程式繼續執行...")

**重點說明**：
- `__enter__`：進入 `with` 區塊時執行，通常用於取得資源
- `__exit__`：離開 `with` 區塊時執行，**保證一定會執行**（即使發生例外）
- `__exit__` 參數：
  - `exc_type`：例外類型（沒有例外時為 None）
  - `exc_value`：例外實例
  - `traceback`：追蹤資訊
- 返回值：
  - `False` 或 `None`：重新拋出例外
  - `True`：抑制例外

# Part III: 本章總結

## 📊 知識回顧

### 1. 特殊方法分類

| 類別 | 方法 | 用途 |
|:-----|:-----|:-----|
| **基本** | `__init__`, `__str__`, `__repr__` | 初始化與表示 |
| **運算** | `__add__`, `__sub__`, `__mul__` | 算術運算子 |
| **比較** | `__eq__`, `__lt__`, `__le__` | 比較運算子 |
| **容器** | `__len__`, `__getitem__`, `__setitem__` | 序列操作 |
| **進階** | `__call__`, `__enter__`, `__exit__` | 特殊功能 |

### 2. 設計原則

✅ **應該**：
- 讓自訂類別行為符合直覺
- 只實作意義明確的特殊方法
- 算術運算子返回新物件（不可變性）
- 優先實作 `__repr__`（可作為 `__str__` 的 fallback）

❌ **不應該**：
- 濫用運算子重載（意義不明確）
- 修改原物件（算術運算應返回新物件）
- 忽略例外處理（上下文管理器）

## ⚠️ 常見誤區

### 誤區 1：混淆 `__str__` 和 `__repr__`
```python
# ❌ 錯誤：兩者內容相同
def __str__(self):
    return f"Person({self.name}, {self.age})"

def __repr__(self):
    return f"Person({self.name}, {self.age})"  # 浪費

# ✅ 正確：分工明確
def __str__(self):
    return f"{self.name}, {self.age} 歲"  # 給使用者

def __repr__(self):
    return f"Person('{self.name}', {self.age})"  # 給開發者
```

### 誤區 2：運算子重載忘記返回值
```python
# ❌ 錯誤：沒有返回值
def __add__(self, other):
    self.x += other.x  # 還修改了原物件！

# ✅ 正確：返回新物件
def __add__(self, other):
    return Vector(self.x + other.x, self.y + other.y)
```

### 誤區 3：上下文管理器忘記處理例外
```python
# ❌ 危險：例外時資源可能沒釋放
def __exit__(self, exc_type, exc_value, traceback):
    self.file.close()  # 如果之前就出錯，file 可能是 None

# ✅ 安全：檢查後再關閉
def __exit__(self, exc_type, exc_value, traceback):
    if self.file:
        self.file.close()
    return False  # 重新拋出例外
```

## ✅ 自我檢核

完成本章後，你應該能夠：

- [ ] 解釋特殊方法的運作原理
- [ ] 區分 `__str__` 與 `__repr__` 的用途
- [ ] 實作算術運算子重載（+、-、*、/）
- [ ] 實作比較運算子（==、<、>等）
- [ ] 使用 `__len__` 和 `__getitem__` 建立序列物件
- [ ] 實作 `__call__` 建立可呼叫物件
- [ ] 實作上下文管理器（`__enter__`、`__exit__`）
- [ ] 判斷何時應該/不應該使用運算子重載

## 🔗 延伸閱讀

### 進階主題
1. **更多特殊方法**：
   - `__setitem__`、`__delitem__`：完整的容器協定
   - `__iter__`、`__next__`：自訂迭代器
   - `__hash__`、`__eq__`：可雜湊物件（字典鍵）

2. **進階技巧**：
   - `functools.total_ordering`：簡化比較方法
   - `contextlib.contextmanager`：用裝飾器建立上下文管理器
   - 描述器協定：`__get__`、`__set__`、`__delete__`

### 實用資源
- [Python 官方文件 - Data Model](https://docs.python.org/3/reference/datamodel.html)
- *Fluent Python* by Luciano Ramalho (Chapter 1, 11, 13)
- [Real Python: Operator Overloading](https://realpython.com/operator-function-overloading/)

## 💪 練習建議

1. **完成課堂練習**（`03-practice.ipynb`）：8 題基礎練習
2. **完成課後習題**（`04-exercises.ipynb`）：12 題進階練習
3. **自我測驗**（`quiz.ipynb`）：檢驗學習成效

**下一步**：前往 `02-worked-examples.ipynb` 查看詳細範例解析！